In [1]:
# Cell 1: Fetch, Store, and Calculate Tokens
!pip install -q google-play-scraper pandas google-generativeai

import pandas as pd
import datetime
import os
from google_play_scraper import Sort, reviews
import google.generativeai as genai
from kaggle_secrets import UserSecretsClient

# --- 1. Configuration ---
APP_ID = 'in.swiggy.android' 
# APP_ID = 'com.application.zomato' # Change this to fetch different apps
DAYS_TO_FETCH = 2
WORKING_DIR = "/kaggle/working"

# Setup Dates
END_DATE = datetime.datetime.now()
START_DATE = END_DATE - datetime.timedelta(days=DAYS_TO_FETCH)

# --- 2. Ingestion Engine ---
print(f"🚀 Fetching reviews for {APP_ID} from {START_DATE.date()} to {END_DATE.date()}...")

all_reviews = []
continuation_token = None

# We fetch a bit more to ensure coverage, then filter
while True:
    result, continuation_token = reviews(
        APP_ID,
        lang='en',
        country='in',
        sort=Sort.NEWEST,
        count=200, 
        continuation_token=continuation_token
    )
    
    if not result: break
    
    # Check if we have passed the start date
    oldest_in_batch = result[-1]['at']
    
    for r in result:
        if r['at'] >= START_DATE:
            all_reviews.append({
                'id': r['reviewId'],
                'date': r['at'],
                'content': r['content'],
                'score': r['score'],
                'app_id': APP_ID
            })
    
    if oldest_in_batch < START_DATE:
        break

# --- 3. Data Storage (Separated by App ID) ---
df = pd.DataFrame(all_reviews)
csv_filename = f"{WORKING_DIR}/{APP_ID}_raw_reviews.csv"
df.to_csv(csv_filename, index=False)

# --- 4. Token Counting & Sampling ---
# Initialize API just for token counting (no cost)
try:
    user_secrets = UserSecretsClient()
    GOOGLE_API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")
    genai.configure(api_key=GOOGLE_API_KEY)
    model = genai.GenerativeModel('gemini-2.5-flash')
    
    # Prepare text payload for counting
    full_text_payload = "\n".join(df['content'].tolist())
    token_count = model.count_tokens(full_text_payload).total_tokens
    
except Exception as e:
    token_count = len(full_text_payload) // 4 # Rough fallback
    print(f"⚠️ API Key Error (Using estimate): {e}")

print(f"\n✅ Ingestion Complete!")
print(f"📂 Saved to: {csv_filename}")
print(f"📊 Total Reviews: {len(df)}")
print(f"🔢 Total Token Count: ~{token_count} tokens")
print("-" * 30)
print("📝 Sample Reviews (First 10):")
display(df[['date', 'content', 'score']].head(10))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.0 MB/s eta 0:00:00
🚀 Fetching reviews for in.swiggy.android from 2026-01-05 to 2026-01-07...

✅ Ingestion Complete!
📂 Saved to: /kaggle/working/in.swiggy.android_raw_reviews.csv
📊 Total Reviews: 447
🔢 Total Token Count: ~5167 tokens
------------------------------
📝 Sample Reviews (First 10):


,date,content,score
0,2026-01-06 08:24:47,worst app don't buy any thing from Instamart.....,1
1,2026-01-06 08:19:23,good,5
2,2026-01-06 08:17:28,Best,5
3,2026-01-06 08:10:32,login problem too much,1
4,2026-01-06 08:10:25,good,3
5,2026-01-06 08:07:26,good,5
6,2026-01-06 08:05:37,good app,5
7,2026-01-06 08:02:52,Worst technical team and support team. Was one...,1
8,2026-01-06 08:01:45,food taste very good 😋,5
9,2026-01-06 07:59:19,total froud worst app.full itam not delivered,1


In [3]:
# Cell 3: Trend Analysis Report (1 Call - Gemini Pro)

# Load Processed Data
df = pd.read_csv(f"{WORKING_DIR}/{APP_ID}_processed_reviews.csv")

# 1. Prepare Statistical Summary
# We don't need to send 1000 reviews again. We send the aggregated data.
topic_counts = df['topic'].value_counts().to_string()
avg_rating_per_topic = df.groupby('topic')['score'].mean().round(2).to_string()

# Get 3 representative examples per topic for context
examples_text = ""
for topic in df['topic'].unique():
    samples = df[df['topic'] == topic]['content'].head(3).tolist()
    examples_text += f"\nTopic: {topic}\nExamples: {samples}\n"

# 2. The Analysis Prompt
analysis_prompt = f"""
You are the Head of Product for this App. 
Based on the data below from the last {DAYS_TO_FETCH} days, write a Strategic Trend Analysis Report.

Data provided:
1. Topic Volume:
{topic_counts}

2. Average Sentiment (1-5 Stars) per Topic:
{avg_rating_per_topic}

3. Verbatim Examples:
{examples_text}

Output Requirements:
1. **Executive Summary**: 2-3 sentences on the overall health.
2. **Critical Issues (Red Flags)**: High volume, low rating areas. What is breaking?
3. **Feature Requests & Feedback**: What do users want?
4. **Action Plan**: 3 concrete steps for the engineering/product team.

Output Format: Markdown.
"""

print("🧠 Calling Gemini 2.5 Pro for Strategic Analysis...")

try:
    # Using Pro for better reasoning
    model_pro = genai.GenerativeModel('gemini-2.5-pro')
    
    analysis_response = model_pro.generate_content(analysis_prompt)
    
    print("\n" + "="*40)
    from IPython.display import Markdown
    display(Markdown(analysis_response.text))
    print("="*40)
    
    # Optional: Save Report
    with open(f"{WORKING_DIR}/{APP_ID}_report.md", "w") as f:
        f.write(analysis_response.text)

except Exception as e:
    print(f"❌ Analysis Failed: {e}")

🧠 Calling Gemini 2.5 Pro for Strategic Analysis...



Of course. Here is the Strategic Trend Analysis Report.

***

### **Strategic Trend Analysis Report: User Feedback (Last 48 Hours)**

**To:** Leadership & Engineering Teams
**From:** Head of Product
**Date:** [Current Date]
**Subject:** Analysis of Recent User Feedback & Immediate Action Plan

---

### 1. Executive Summary

While we see a high volume of generic positive feedback, our core service delivery is facing a crisis. Critical operational areas such as delivery, order accuracy, and customer support are failing, resulting in extremely low sentiment scores (averaging 1.1-1.6 out of 5) and creating significant user frustration. These issues are actively eroding user trust and require immediate, focused intervention to prevent churn and reputational damage.

### 2. Critical Issues (Red Flags)

Our analysis highlights three interconnected areas that are severely underperforming. These are not isolated complaints but patterns indicating systemic problems.

*   **Core Fulfillment Failure (Delivery & Accuracy):**
    *   **Data:** "Delivery Delays" is our second-highest specific complaint category (35 mentions) with a sentiment of 1.40/5. "Order Accuracy/Missing Items" follows closely with 21 mentions and a 1.67/5 sentiment.
    *   **Analysis:** Our fundamental promise of delivering the correct items on time is broken. Verbatims mention "constant delays," "ETA that changes in every minute," and receiving "expired and cheap quality products." The fact that even premium "Swiggy Black" members are experiencing this indicates a deep operational issue, not just an edge case. This is our most significant business risk.

*   **Support System Breakdown:**
    *   **Data:** "Customer Support Issues" has the lowest sentiment score across all topics (**1.14/5**) with a high volume of 28 mentions.
    *   **Analysis:** Our support system is actively making bad situations worse. Users report that their issues are "not resolved" and describe the service as the "worst." This means that when our core fulfillment fails, users have no effective recourse, turning a single bad experience into a reason to abandon the app entirely.

*   **Trust & Transparency Gaps (Pricing & Payments):**
    *   **Data:** "Pricing & Fees" has a critically low sentiment of 1.36/5. "Payment Method Issues" also shows problems, with verbatims citing un-refunded failed payments.
    *   **Analysis:** Users feel cheated. Complaints about high delivery charges are compounded by reports of being "charged more money after the order was placed." Furthermore, the failure to refund money from failed transactions is a cardinal sin in e-commerce, causing irreparable damage to user trust.

### 3. Feature Requests & Feedback

Beneath the frustration, users are telling us what they need to see improved:

*   **Reliable Issue Resolution:** Users want a simple, in-app process to report missing, wrong, or expired items with photo/video evidence. The current system is perceived as a dead end.
*   **Transparent Checkout:** Users are demanding clarity on costs. The checkout screen must explicitly break down all charges (item cost, delivery fee, taxes, etc.) *before* the payment is confirmed to eliminate surprises.
*   **Accurate Delivery ETAs:** The fluctuating ETA is a major source of frustration. Users need a reliable and accurate real-time tracking experience.
*   **Core App Stability:** Critical bugs, such as the "minimum amount in cart page shows 1999" and login problems, are blocking basic usability and must be fixed.

### 4. Action Plan

Based on this analysis, I am directing our teams to prioritize the following three initiatives immediately:

1.  **Form a "Fulfillment Task Force" (Lead: Ops & Product):**
    *   **Action:** Assemble a cross-functional team (Product, Ops, Eng) to map the entire order lifecycle from placement to delivery.
    *   **Objective:** Identify the top 3 root causes of delays and inaccuracies within one week. Propose and begin implementing operational (e.g., picker training, inventory checks) and technical (e.g., inventory sync logic, ETA algorithm refinement) fixes in the next sprint.

2.  **Prioritize a "Self-Serve Resolution Flow" (Lead: Product & Eng):**
    *   **Action:** Immediately scope and begin development of an enhanced in-app "Report an Issue" feature.
    *   **Objective:** In the next 2-3 sprints, deliver a flow where users can select specific items from an order, report a problem (missing, damaged, expired), upload photo evidence, and trigger an automated refund or credit. This will provide immediate resolution for users and reduce the load on our failing support channels.

3.  **Launch a "Trust & Stability" Sprint (Lead: Eng & Design):**
    *   **Action:** Dedicate the next engineering sprint to fixing critical financial and trust-related issues.
    *   **Objective:** a) Resolve the "payment failed, money deducted" bug as the #1 priority. b) Fix the cart value and login bugs. c) Redesign the checkout UI to provide a crystal-clear, itemized cost breakdown before a user confirms their order.

In [ ]:
# Cell 2: High-Performance Parallel Classification
import json
import time
import math
import pandas as pd
import google.generativeai as genai
from concurrent.futures import ThreadPoolExecutor, as_completed

# Load Data
try:
    df = pd.read_csv(f"{WORKING_DIR}/{APP_ID}_raw_reviews.csv")
    print(f"📂 Loaded {len(df)} reviews.")
except FileNotFoundError:
    print("❌ Error: Run Cell 1 first to fetch data.")
    df = pd.DataFrame()

# --- CONFIGURATION ---
DISCOVERY_SAMPLE_SIZE = 100 
BATCH_SIZE = 50           # Smaller batches are safer for parallel execution
MAX_WORKERS = 8           # Number of simultaneous API calls (Parallel threads)

model_flash = genai.GenerativeModel('gemini-2.5-flash')

# ==========================================
# PASS 1: TAXONOMY DISCOVERY (Serial - Fast)
# ==========================================
print("\n🔍 PASS 1: Discovering Taxonomy...")
TAXONOMY = []

if not df.empty:
    sample_reviews = df['content'].sample(min(DISCOVERY_SAMPLE_SIZE, len(df)), random_state=42).tolist()
    sample_text = "\n".join([f"- {r[:150]}" for r in sample_reviews])

    discovery_prompt = f"""
    Analyze these user reviews for a food delivery app.
    Identify the Top 6 distinct categories.
    Output ONLY a Python list of strings.
    Reviews: {sample_text}
    Example Output: ["Category A", "Category B"]
    """
    
    try:
        response = model_flash.generate_content(discovery_prompt)
        cleaned_text = response.text.replace('```python', '').replace('```', '').replace('\n', '').strip()
        TAXONOMY = eval(cleaned_text)
        if "Uncategorized" not in TAXONOMY: TAXONOMY.append("Uncategorized")
        print(f"✅ Taxonomy Locked: {TAXONOMY}")
    except:
        TAXONOMY = ["Delivery Issues", "App Bugs", "Food Quality", "Refunds", "Positive Feedback", "Uncategorized"]
        print("⚠️ Used Default Taxonomy.")

    # ==========================================
    # PASS 2: PARALLEL BATCH CLASSIFICATION
    # ==========================================
    print(f"\n🚀 PASS 2: Classifying {len(df)} reviews (Parallel Threads)...")
    start_time = time.time()
    
    # 1. Define the Worker Function
    def process_batch(batch_df, batch_index):
        batch_text = ""
        for _, row in batch_df.iterrows():
            # Clean and minify text
            clean_text = str(row['content']).replace('\n', ' ').replace('"', "'")[:200]
            batch_text += f"{row['id']}: {clean_text}\n"
            
        prompt = f"""
        Map each Review ID to exactly ONE category from: {TAXONOMY}
        Output strict JSON: {{"id": "category"}}
        Reviews:
        {batch_text}
        """
        
        try:
            # Add retries for stability
            for attempt in range(3):
                try:
                    resp = model_flash.generate_content(
                        prompt, 
                        generation_config={"response_mime_type": "application/json"}
                    )
                    return json.loads(resp.text)
                except Exception as e:
                    if "429" in str(e): # Rate limit hit
                        time.sleep(2 * (attempt + 1)) # Backoff
                        continue
                    raise e
        except Exception as e:
            print(f"   ❌ Batch {batch_index} Failed: {e}")
            return {rid: "Uncategorized" for rid in batch_df['id']}

    # 2. Split Data into Chunks
    batches = []
    num_batches = math.ceil(len(df) / BATCH_SIZE)
    for i in range(num_batches):
        batches.append((df.iloc[i*BATCH_SIZE : (i+1)*BATCH_SIZE], i))

    # 3. Execute in Parallel
    review_category_map = {}
    completed_count = 0
    
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        # Submit all tasks
        future_to_batch = {executor.submit(process_batch, b_df, idx): idx for b_df, idx in batches}
        
        # Process as they complete
        for future in as_completed(future_to_batch):
            batch_result = future.result()
            review_category_map.update(batch_result)
            completed_count += 1
            print(f"\r   ⚡ Progress: {completed_count}/{num_batches} batches done...", end="")

    # ==========================================
    # MERGE & SAVE
    # ==========================================
    df['topic'] = df['id'].map(review_category_map).fillna("Uncategorized")
    
    processed_filename = f"{WORKING_DIR}/{APP_ID}_processed_reviews.csv"
    df.to_csv(processed_filename, index=False)
    
    elapsed = time.time() - start_time
    print(f"\n\n✅ Done in {elapsed:.2f} seconds!")
    print(f"📊 Speed: {len(df) / elapsed:.1f} reviews/sec")
    print(df['topic'].value_counts().head())

else:
    print("No data.")

📂 Loaded 447 reviews.

🔍 PASS 1: Discovering Taxonomy...
✅ Taxonomy Locked: ['Overall App Experience & Satisfaction', 'Delivery Speed & Timeliness', 'Customer Support & Issue Resolution', 'Order Accuracy & Product Quality', 'Pricing & Delivery Charges', 'Payment Options & COD Availability', 'Uncategorized']

🚀 PASS 2: Classifying 447 reviews (Parallel Threads)...
   ⚡ Progress: 9/9 batches done...

✅ Done in 81.47 seconds!
📊 Speed: 5.5 reviews/sec
topic
Overall App Experience & Satisfaction    293
Delivery Speed & Timeliness               40
Customer Support & Issue Resolution       38
Order Accuracy & Product Quality          28
Uncategorized                             21
Name: count, dtype: int64
